# Install libraries

In [1]:
pip install folium


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install shapely 


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


# Import Libraries

In [4]:
import pandas as pd
file_name = 'data.xlsx' 
df = pd.read_excel(file_name, index_col=0)
import folium

In [5]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import geopandas as gpd
import shapely.wkt
%matplotlib inline

In [6]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

# Organize the data into Mother and daughter and assign serial names to depots

In [7]:
m = np.asarray(df.index[df['Category'] == "Daughter"].tolist())
cluster_labels = m

# get the number of clusters
num_clusters = len(set(cluster_labels))

In [8]:
points = df[df['Category'] == "Daughter"]
stations = df[df['Category'] == "Mother"]

In [9]:
for i in range(len(stations)):
    stations['Category'].iloc[i]='Mother'+str(i)

/Users/vbhattac/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/vbhattac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Visualize the map 

In [10]:
def create_gdf(df, x='latitude', y='longitude'):
    return gpd.GeoDataFrame(df, 
    geometry=gpd.points_from_xy(df[y], df[x]), 
             crs={'init':'epsg:4326'})
stations_gdf = create_gdf(stations)
points_gdf = create_gdf(points)

In [11]:
locs_stations = zip(stations_gdf['latitude'], stations_gdf['longitude'])
locs_points = zip(points_gdf['latitude'], points_gdf['longitude'])

a = df['latitude'][1]
b = df['longitude'][1]
m = folium.Map(
    location=[a,b],
    tiles='CartoDb dark_matter',
    zoom_start=10,
)

for location in locs_stations:
    folium.CircleMarker(location=location, 
        color='red',   radius=4).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, 
        color='white', radius=1).add_to(m)
m

# Calculate nearest neighbours

In [12]:
def calculate_nearest(row, destination, val, col='geometry'):
    # 1 - create unary union    
    dest_unary = destination['geometry'].unary_union
    # 2 - find closest point
    nearest_geom = nearest_points(row[col], dest_unary)
    # 3 - Find the corresponding geom
    match_geom = destination.loc[destination.geometry 
                == nearest_geom[1]]
    # 4 - get the corresponding value
    match_value = match_geom[val].to_numpy()[0]
    return match_value

In [14]:
from shapely.ops import nearest_points
# Get the nearest geometry
points_gdf['nearest_geom'] = points_gdf.apply(calculate_nearest, destination=stations_gdf, val='geometry', axis=1)
# Get the nearest Bike station name
points_gdf['nearest_station'] = points_gdf.apply(calculate_nearest, destination=stations_gdf, val='Category', axis=1)
points_gdf


# Get the list of depots 

In [15]:
list_m = points_gdf['nearest_station'].unique()

print(list_m)

['Mother3' 'Mother2' 'Mother4' 'Mother10' 'Mother1' 'Mother8' 'Mother0'
 'Mother9' 'Mother6' 'Mother7' 'Mother5']


# Solve the routing problem

In [16]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from sklearn.neighbors.dist_metrics import DistanceMetric
dist = DistanceMetric.get_metric('haversine')

## function to calculate cost for route
def calculate_cost(distance,average_speed):
    cost = []
    if distance < 100:
        cost = 500
    elif distance > 100 and distance/average_speed < 6 :
        cost = 500 + (distance%100)*10
    else:
        cost = 500 + (distance%100)*10 + 50
    return cost 

#function to stores the data for the problem.
def create_data_model(num,number_of_trucks):
    data = {}
    data['distance_matrix'] = num
    data['num_vehicles'] = number_of_trucks
    data['depot'] = 0
    return data

#function to print the solution for the problem.
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    average_speed = 30
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}km\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
        print('Cost of travel: {}INR'.format(calculate_cost(total_distance,30)))
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}km'.format(max_route_distance))


def main(list_m):
    for i in range(len(list_m)):
        stations_sub = points_gdf[points_gdf['nearest_station'] == list_m[i]]
        mn = pd.DataFrame() 
        mn['latitude']=stations_sub['latitude']
        mn['longitude']=stations_sub['longitude']
        temp = stations.loc[stations['Category'] == list_m[i]]
        new_row = pd.DataFrame({'latitude':temp.iloc[0]['latitude'], 'longitude':temp.iloc[0]['longitude']},index =[1]) 
        mn = pd.concat([new_row, mn]).reset_index(drop = True) 
        aa=pd.DataFrame(dist.pairwise(mn[['latitude','longitude']].to_numpy())*6373, index=mn.index, columns=mn.index) 
        # Instantiate the data problem.
        num = aa.reset_index().values
        number_of_trucks = 1 #set the number of trucks to 1
        data = create_data_model(num,number_of_trucks)
        # Create the routing index manager.
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),data['num_vehicles'], data['depot'])

        # Create Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # Create and register a transit callback.
        def distance_callback(from_index, to_index):
        # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # Add Distance constraint.
        dimension_name = 'Distance'
        routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        120,  # vehicle maximum travel distance -> . from travel time of 4 hrs * cruise set speed 30km/hr
        True,  # start cumul to zero
        dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # Solve the problem.
        solution = routing.SolveWithParameters(search_parameters)

        # Print solution on console.
        if solution:
            print_solution(data, manager, routing, solution)


In [17]:
main(list_m)    


Route for vehicle 0:
 0 ->  1 ->  2 ->  3 ->  4 ->  5 ->  6 ->  7 ->  8 ->  9 ->  10 ->  11 ->  12 ->  13 ->  14 ->  15 ->  16 ->  17 ->  18 ->  19 ->  20 ->  21 ->  22 ->  23 -> 0
Distance of the route: 23km

Cost of travel: 500INR
Maximum of the route distances: 23km
Route for vehicle 0:
 0 ->  1 ->  2 ->  3 ->  4 ->  5 ->  6 ->  7 ->  8 ->  9 ->  10 ->  11 ->  12 ->  13 ->  14 ->  15 ->  16 ->  17 ->  18 ->  19 ->  20 ->  21 ->  22 ->  23 ->  24 ->  25 -> 0
Distance of the route: 25km

Cost of travel: 500INR
Maximum of the route distances: 25km
Route for vehicle 0:
 0 ->  1 ->  2 ->  3 ->  4 ->  5 ->  6 ->  7 ->  8 ->  9 ->  10 ->  11 ->  12 ->  13 ->  14 ->  15 ->  16 -> 0
Distance of the route: 16km

Cost of travel: 500INR
Maximum of the route distances: 16km
Route for vehicle 0:
 0 ->  26 ->  27 ->  28 ->  29 ->  30 ->  31 ->  32 ->  14 ->  15 ->  16 ->  17 ->  18 ->  19 ->  20 ->  21 ->  22 ->  23 ->  24 ->  25 ->  1 ->  2 ->  3 ->  4 ->  5 ->  6 ->  7 ->  8 ->  9 ->  10 ->  11 

# Display the clusters in the map and save them to CSV

In [18]:
from shapely.geometry import Point, LineString
# Create LineString Geometry
points_gdf['line'] = points_gdf.apply(lambda row: LineString([row['geometry'], row['nearest_geom']]), axis=1)
# Create Line Geodataframe
line_gdf = points_gdf[["nearest_station", "line"]].set_geometry('line')
# Set the Coordinate reference
line_gdf.crs = crs={"init":"epsg:4326"}

In [20]:
locs_stations = zip(stations_gdf['latitude'], stations_gdf['longitude'])
locs_points = zip(points_gdf['latitude'], points_gdf['longitude'])

a = df['latitude'][1]
b = df['longitude'][1]
m = folium.Map(
    location=[a,b],
    tiles='CartoDb dark_matter',
    zoom_start=10,
)
points_gdf.to_csv('out.csv')

for location in locs_stations:
    folium.CircleMarker(location=location, 
        color='red',   radius=4).add_to(m)
for location in locs_points:
    folium.CircleMarker(location=location, 
        color='white', radius=1).add_to(m)
folium.GeoJson(line_gdf).add_to(m)
m
